An analysis of the journals affiliated to the Digital-Humanities discipline according to "Preliminary results on mapping digital humanities research" article. 

In [155]:
import requests
import json
import copy

class Journals(object):
    __csv_path = ""
    __journals = []
    
    #Let's check all the citations in Crossref for each of the following journals
    #I will use this API call: https://api.crossref.org/journals/<ISSN>/works
    CROSSREF_CALL = "https://api.crossref.org/journals/%s/works"
    
    def __init__(self, csv_path):
        self.__csv_path = csv_path
    
    def get_journals(self):
        return self.__journals
    
    def build_obj_from_csv(self):
        # import all the Dh Journals according to the work made by Gianmarco et.al 
        journals_csv = open(self.__csv_path)
        csv_reader = csv.reader(journals_csv, delimiter=',')
        # This skips the first row of the CSV file: the header.
        #'ID', 'E_ISSN', 'P_ISSN', 'TITLE', 'URL', 'level', 'DH LEVEL', 'Iteration'
        next(csv_reader)
        
        for row in csv_reader:
            a_journal = {}
            a_journal['ID'] = row[0]
            a_journal['E_ISSN'] = row[1]
            a_journal['P_ISSN'] = row[2]
            a_journal['TITLE'] = row[3]
            a_journal['URL'] = row[4]
            a_journal['level'] = row[5]
            a_journal['DH LEVEL'] = row[6]
            a_journal['Iteration'] = row[7]
            self.__journals.append(a_journal)
            
            
    def crossref_req_call(self, call, issn_value, rows, offset):
        params = "?rows="+str(rows)+"&offset="+str(offset)
        req_call = call%issn_value
        response = requests.get(req_call+params)
        list_of_dois = []
        print(req_call+params)
        
        if response.status_code == 200:
            json_data = json.loads(response.content)
            print("  -> Found: "+str(len(json_data['message']["items"]))+"#")
            for an_item in json_data['message']["items"]:
                if "DOI" in an_item:
                    list_of_dois.append(an_item["DOI"])
            
            if len(json_data['message']["items"]) != 0:
                self.crossref_req_call(call, issn_value, rows, offset + rows)
        else:
            return False
        
        return list_of_dois
        
    
    def extend_journals_with_dois(self, SERVICE_CALL = "", LIMIT = 0):
        
        ROWSXPAGE = 500
        REQ_CALL = ""
        if SERVICE_CALL == "crossref":
            REQ_CALL = self.CROSSREF_CALL
        
        LIMIT_LOOP = LIMIT
        extended_journals = copy.deepcopy(dh_journals)
        for a_journal_obj in extended_journals:

            #try both ISSNs
            for issn_attr in ["E_ISSN","P_ISSN"]:

                a_journal_obj[issn_attr] = {"value": a_journal_obj[issn_attr], "in": False, "dois":[]}

                issn_value_parts = a_journal_obj[issn_attr]["value"].split(";")
                for issn_value in issn_value_parts:
                    if (issn_value != "") and (issn_value != None):

                        res = self.crossref_req_call(REQ_CALL, issn_value, ROWSXPAGE, 0) 
                        if res != False:
                            a_journal_obj["in"] = True
                            a_journal_obj[issn_attr]["dois"] = res

            #COMMENT this after testitng
            LIMIT_LOOP = LIMIT_LOOP - 1
            if LIMIT_LOOP == 0:
                break
        
        self.__journals = extended_journals

___

In [156]:
journals = Journals("../../../data/phd_thesis/dh_journals.csv")
journals.build_obj_from_csv()

In [159]:
journals.extend_journals_with_dois("crossref",5)

https://api.crossref.org/journals/2532-8816/works?rows=100&offset=0
https://api.crossref.org/journals/2297-2668/works?rows=100&offset=0
  -> Found: 83#
https://api.crossref.org/journals/2297-2668/works?rows=100&offset=100
  -> Found: 0#
https://api.crossref.org/journals/2055-768X/works?rows=100&offset=0
  -> Found: 100#
https://api.crossref.org/journals/2055-768X/works?rows=100&offset=100
  -> Found: 100#
https://api.crossref.org/journals/2055-768X/works?rows=100&offset=200
  -> Found: 100#
https://api.crossref.org/journals/2055-768X/works?rows=100&offset=300
  -> Found: 100#
https://api.crossref.org/journals/2055-768X/works?rows=100&offset=400
  -> Found: 67#
https://api.crossref.org/journals/2055-768X/works?rows=100&offset=500
  -> Found: 0#
https://api.crossref.org/journals/2055-7671/works?rows=100&offset=0
  -> Found: 100#
https://api.crossref.org/journals/2055-7671/works?rows=100&offset=100
  -> Found: 100#
https://api.crossref.org/journals/2055-7671/works?rows=100&offset=200
  ->

In [160]:
journals.get_journals()

[{'ID': '1',
  'E_ISSN': {'value': '2532-8816', 'in': False, 'dois': []},
  'P_ISSN': {'value': '', 'in': False, 'dois': []},
  'TITLE': 'Umanistica Digitale',
  'URL': 'https://umanisticadigitale.unibo.it/',
  'level': '1',
  'DH LEVEL': 'Exclusively',
  'Iteration': '1'},
 {'ID': '2',
  'E_ISSN': {'value': '2297-2668',
   'in': False,
   'dois': ['10.3389/fdigh.2015.00005',
    '10.3389/fdigh.2015.00008',
    '10.3389/fdigh.2019.00002',
    '10.3389/fdigh.2014.00001',
    '10.3389/fdigh.2016.00001',
    '10.3389/fdigh.2016.00004',
    '10.3389/fdigh.2019.00009',
    '10.3389/fdigh.2016.00007',
    '10.3389/fdigh.2016.00009',
    '10.3389/fdigh.2016.00008',
    '10.3389/fdigh.2017.00002',
    '10.3389/fdigh.2017.00005',
    '10.3389/fdigh.2017.00007',
    '10.3389/fdigh.2018.00002',
    '10.3389/fdigh.2018.00003',
    '10.3389/fdigh.2018.00004',
    '10.3389/fdigh.2017.00011',
    '10.3389/fdigh.2018.00014',
    '10.3389/fdigh.2017.00015',
    '10.3389/fdigh.2018.00015',
    '10.3389/

In [166]:
s = "\nbcdefg"

In [167]:
s[1:]

'bcdefg'